In [12]:
pip install gensim

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 67.1/67.1 kB ? eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20448 sha256=88aee6637f468ee5a32fad43d7609f4bb1998ee104ee35a2151b6cbc1f246bfe
  Stored in directory: c:\users\adarsh\appdata\local\pip\cache\wheels\01\02\ee\df0699282986903a384b69aab4413af9efd26b3612b5dccc9e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3522 sha256=03885df31fd18e8993b542a239b9c2016a15a6014d9ab344bc3af602a38201d8
  Stored in directory: c:\users\adarsh\appdata\local\pip\cache\wheels\43\aa\48\5c66b931ff013ad19774081aa19656637af5c0cc33b5494b30
Successfully built fst-pso miniful
Note: you may need to restart the kernel to use

In [13]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from PIL import Image
#from pywaffle import Waffle
#from catboost import CatBoostClassifier, pool
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from tqdm import tqdm
import nltk
from nltk.stem import WordNetLemmatizer
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize

In [2]:
import nltk
nltk.download('omw-1.4')
# Check if WordNet data is available and download if not
try:
    nltk.data.find('corpora/wordnet')
except LookupError:
    nltk.download('wordnet', download_dir='/kaggle/working/nltk_data')

# Update the NLTK data path to include the WordNet data
nltk.data.path.append('/kaggle/working/nltk_data')

# Check if stopwords data is available and download if not
try:
    nltk.data.find('corpora/stopwords')
except LookupError:
    nltk.download('stopwords', download_dir='/kaggle/working/nltk_data')

# Update the NLTK data path to include the stopwords data
nltk.data.path.append('/kaggle/working/nltk_data')


    
from nltk.corpus import stopwords

import re
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score,roc_auc_score,roc_curve
from sklearn.naive_bayes import MultinomialNB

from sklearn.ensemble import HistGradientBoostingClassifier

import plotly.express as px
import warnings


from sklearn.neighbors import KNeighborsClassifier

from imblearn.over_sampling import SMOTE

import squarify
from sklearn.metrics import f1_score


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Adarsh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /kaggle/working/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
from nltk.corpus import wordnet
sy =wordnet.synsets('hello')

In [6]:
synsets = wordnet.synsets('extrovert')

# Print synset information
for synset in synsets:
    print(f"Synset name: {synset.name()}")
    print(f"Definition: {synset.definition()}")
    print(f"Examples: {synset.examples()}")
    print()

Synset name: extrovert.n.01
Definition: (psychology) a person concerned more with practical realities than with inner thoughts and feelings
Examples: []

Synset name: extrovert.s.01
Definition: being concerned with the social and physical environment
Examples: []



In [7]:
def sampling(mode, df, col_name, class_names, count):
    """
    Samples the dataframe based on the mode, given class names and count.

    Parameters:
    -----------
    mode: str
        Either 'under' or 'over'
    df : pandas.DataFrame
        Input DataFrame to be undersampled
    col_name : str
        Name of the column containing the target class
    class_names : list
        List of target class names to be undersampled
    count : int
        Number of samples to be kept for each class

    Returns:
    --------
    pandas.DataFrame
        Undersampled DataFrame or Oversampled
    """
    dfs = []
    for class_name in class_names:
        if mode == 'under':
            df_class = df[df[col_name] == class_name]
            if len(df_class) > count:
                df_class = df_class.sample(count, replace=False)
            dfs.append(df_class)
        elif mode == 'over':
            df_class = df[df[col_name] == class_name]
            if len(df_class) < count:
                df_class = df_class.sample(count, replace=True)
            dfs.append(df_class)
    dfs = pd.concat(dfs)
    df_x = df[~df[col_name].isin(class_names)]
    return pd.concat([dfs,df_x])

In [15]:
def get_train(train_data):
    
    X_train = np.vstack(np.array(train_data.posts))
    y_train = np.array(train_data.type)
    
    return X_train, y_train

In [16]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

def clear_text(df):
    lemmatizer = WordNetLemmatizer()
    cleaned_posts = []
    for post in df.posts:
        # Remove special characters and digits
        post = re.sub(r"[^a-zA-Z]", " ", post)
        # Tokenize and lemmatize words
        words = word_tokenize(post.lower())
        words = [lemmatizer.lemmatize(word) for word in words]
        # Remove stopwords
        stop_words = set(stopwords.words("english"))
        words = [word for word in words if word not in stop_words]
        cleaned_posts.append(" ".join(words))
    return cleaned_posts, len(cleaned_posts)


In [ ]:
def split(df, size):
    # Cleaning The Data
    df.posts, length = clear_text(df)

    # Splitting into train & test
    print("Splitting into train & test")
    train_data, test_data = train_test_split(df, test_size=size, random_state=0, stratify=df.type)

    # Applying Word2Vec
    print("Applying Word2Vec")
    tokenized_posts = [word_tokenize(post) for post in train_data.posts]
    model = Word2Vec(tokenized_posts, vector_size=100, window=5, min_count=1, workers=4)

    # Getting the average word embeddings for each post in the training set
    train_post = [sum([model.wv[word] for word in words]) / len(words) for words in tokenized_posts]
    train_post = np.array(train_post)

    # Applying the same Word2Vec model to the test set
    tokenized_test_posts = [word_tokenize(post) for post in test_data.posts]
    test_post = [sum([model.wv[word] for word in words]) / len(words) for words in tokenized_test_posts]
    test_post = np.array(test_post)

    # Label Encoding the classes as 0, 1, 2, 3......
    print("Label Encoding the classes")
    target_encoder = LabelEncoder()

    # Getting the final train and test
    print("Getting the final train and test")
    train_target = target_encoder.fit_transform(train_data.type)
    test_target = target_encoder.fit_transform(test_data.type)
    print(target_encoder.classes_)
    return train_post, test_post, train_target, test_target
